In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# Load your data
df = pd.read_csv("dataset.csv")

# Drop 'Unnamed: 0' column if it exists
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

# Separate features and labels
X = df.drop('label', axis=1)
y = df['label']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle imbalance with SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_scaled, y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# ---------- Base Models ----------

# Isolation Forest
iso_model = IsolationForest(contamination=0.01, random_state=42)
iso_model.fit(X_train)
iso_preds = iso_model.predict(X_test)
iso_preds = np.where(iso_preds == -1, 1, 0)

# One-Class SVM
svm_model = OneClassSVM(nu=0.01, kernel='rbf', gamma=0.1)
svm_model.fit(X_train)
svm_preds = svm_model.predict(X_test)
svm_preds = np.where(svm_preds == -1, 1, 0)

# Local Outlier Factor
lof_model = LocalOutlierFactor(n_neighbors=20, contamination=0.01)
lof_preds = lof_model.fit_predict(X_test)
lof_preds = np.where(lof_preds == -1, 1, 0)

# Stack predictions as features
X_stack = np.vstack((iso_preds, svm_preds, lof_preds)).T

# ---------- Meta-Model: Neural Network ----------
nn_model = MLPClassifier(hidden_layer_sizes=(50, 30), activation='relu', solver='adam', max_iter=1000, random_state=42)
nn_model.fit(X_stack, y_test)
stacked_preds = nn_model.predict(X_stack)

# ---------- Evaluation ----------
accuracy = accuracy_score(y_test, stacked_preds)
precision = precision_score(y_test, stacked_preds)
recall = recall_score(y_test, stacked_preds)
f1 = f1_score(y_test, stacked_preds)
roc_auc = roc_auc_score(y_test, stacked_preds)
conf_matrix = confusion_matrix(y_test, stacked_preds)

print("\n🌟 Classification Report:\n", classification_report(y_test, stacked_preds))
print("🧊 Confusion Matrix:\n", conf_matrix)
print(f"🎯 Accuracy:   {accuracy * 100:.2f}%")
print(f"💖 Precision:  {precision * 100:.2f}%")
print(f"🔥 Recall:     {recall * 100:.2f}%")
print(f"✨ F1-Score:   {f1 * 100:.2f}%")
print(f"🌈 ROC-AUC:    {roc_auc * 100:.2f}%")


In [ ]:
pip install imbalanced-learn